# **MISSION**
1. Discover which countries supply which minerals.
2. Find the chemical element that is most common.
3. which mineral is the oldest?
# Table of Contents
 1. Observe the dataset
 2. Read and Analyse Data
 3. Data Preparation
 4. Data Visualization
 5. Summary

# 1. Observe the dataset

> 1.1 Import necessary modules

In [2]:
#Load packages
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,OrdinalEncoder,LabelBinarizer
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from sklearn.feature_selection import RFE,SelectFromModel
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
plt.style.use('Solarize_Light2')


> 1.2 Read and understand data

* Mineral Name = common name (Pyroxferroite, Gerhardtite, Hydroxylherderite, etc. Silver, Gold and Carbon are elements that form minerals on their own)
* RRUFF Chemistry (plain) = RRUFF formula (Stoiberite = Cu2+5O2(V5+O4)2 which stands for Cu²⁺₅O₂(V⁵⁺O₄)₂ formula)
* IMA Chemistry (plain) = International Mineralogical Association formula (Stoiberite = Cu5O2(VO4)2 which stands for Cu₅O₂(VO₄)₂ formula)
* Chemistry Elements = chemistry elements in mineral (Stoiberite = Cu, V, O, Silver = Ag)
* IMA Number = unique IMA number (if applies) (Stoiberite = IMA1979-016)
* RRUFF IDs = unique RRUFF ID (if applies) - one or more (Silver = R070416, R070463, R070754)
* Country of Type Locality = country (including 'unknown' and '?')
* Year First Published = year the mineral was first officially described
* IMA Status = official status of IMA assigned (Approved, Grandfathered, Pending publication)
* Structural Groupname = structural group of minerals (Platarsite = Pyrite)
* Fleischers Groupname = structural group for minerals by Fleischers Glossary 2008 (corresponds with Structural Groupname)
* Status Notes = publication
* Crystal Systems = a set of point groups and their corresponding space groups are assigned to a lattice system (monoclinic, cubic, orthorhombic, hexagonal, etc)
* Oldest Known Age (Ma) = age in megaannums (one million years)

In [3]:
minerals = pd.read_csv("/kaggle/input/ima-database-of-mineral-properties/RRUFF_Export_20191025_022204.csv")
minerals.sample(5)

# 2. Read and Analyse Data

In [4]:
minerals.info()
# calculate duplicates
dups = minerals.duplicated()
# report if there are any duplicates
print(dups.any())
# list all duplicate rows

print("Duplicate Rows",minerals[dups])
#Cardinality 
minerals.nunique() # To determine the maximum and minimum number of variations in each column of the dataset
#Lets now check for null fields
import seaborn as sns
plt.figure(figsize=(6,6))
sns.heatmap(minerals.isnull(),yticklabels=False,cbar=False,cmap='viridis')
minerals.isnull().sum()


> Obseravation

* **IMA Number**: not important for solution
* **RRUFF IDs**: not important for solution
* **IMA Chemistry**: important for solution
* **Country of Type Locality**: important for solution
* **Oldest Known Age (Ma)**: important for solution

# 3. Data Preparation

> Solution

*  Country
1. Fill NaN's Country with Unknown
2. Strip, split and rename countries
* Elements
1. Remove NaN values in Elements
2. Separate each chemistry element
* Age Fill NaN's Age with Unknown
* Find duplicates
* Rename columns

In [5]:
# 1) Duplicates
dupli = minerals.duplicated()
minerals[dupli].shape
# 2) Columns rename
minerals.rename(columns={'Mineral Name': 'Mineral', 'RRUFF Chemistry (plain)': 'RRUFF Chem', 'IMA Chemistry (plain)': 'IMA Chem',
                         'Chemistry Elements': 'Elements', 'IMA Number': 'IMA ID', 'RRUFF IDs': 'RRUFF ID', 
                         'Country of Type Locality': 'Country', 'Oldest Known Age (Ma)': 'Age (Ma)'}, inplace=True)
print('Done')

In [6]:
# Changing NaN to unknown
minerals['Country'] = minerals['Country'].fillna(value='Unknown') #  Changing the NaN values to unknown
minerals['Country'].isnull().sum()
# Spliting, striping and renaming countries
# Edited from Daria Chemkaeva, (2020).
countries_splited = minerals.set_index('Mineral')['Country'].str.split(' / ', expand=True).stack().reset_index('Mineral').reset_index(drop=True)
countries_splited.columns = ['Mineral', 'Country']
new_met = countries_splited['Country'].str.contains('meteorite', case=False), 'Country'
new_cou = countries_splited['Country'].str.contains('IDP', case=False), 'Country'
countries_splited.loc[new_met] = 'Meteorite'
countries_splited.loc[new_cou] = 'IDP'
countries_splited['Country'] = countries_splited['Country'].replace({' \?':''}, regex=True)
countries_splited['Country'] = countries_splited['Country'].replace('?', 'Unknown')
print(countries_splited[countries_splited['Country'].str.contains('Unknown', regex=False)])
countries_splited

In [7]:
# Percent of data missing
missing_values_count = countries_splited[countries_splited['Country'].str.contains('Unknown', regex=False)].value_counts()
total_cells = np.product(minerals['Country'].shape)
total_missing = missing_values_count.sum()
percent_missing_country = (total_missing/total_cells) * 100
print(percent_missing_country)
# Geting uniques values from Country
countries_splited['Country'].value_counts()
# Dropping Unknown rows from countries_splited
countries_splited.drop(countries_splited[countries_splited['Country'] == 'Unknown'].index, inplace=True)
countries_splited

In [8]:
# Elements
# Displaying rows with NaN values
null_elements = minerals['Elements'].isnull()
minerals[null_elements]
missing_values_count = minerals['Elements'].isnull().sum()
total_cells = np.product(minerals['Elements'].shape)
total_missing = missing_values_count.sum()
percent_missing_elems = (total_missing/total_cells) * 100
print(percent_missing_elems)

In [9]:
# Removing NaN-bearing rows
del_rows = minerals['Elements'].notna()
minerals = minerals[del_rows] #  New DF without the 4 NaN rows
minerals['Elements'].isna().sum()
# Spliting chemistry elements

# Edited from Daria Chemkaeva, (2020).
elements_splited = minerals.set_index('Mineral')['Elements'].str.split(' ', expand=True).stack().reset_index('Mineral').reset_index(drop=True)
elements_splited.columns = ['Mineral', 'Element']
elements_splited

In [10]:
# Age (Ma)
#  Checking unique values
minerals['Age (Ma)'].unique()
# Changing type to float
minerals['Age (Ma)'] = minerals['Age (Ma)'].astype('float64')
print('Done')
# Filling NaNs with Unknown
minerals['Age (Ma)'] = minerals['Age (Ma)'].replace(0, 'Unknown')
minerals['Age (Ma)']
minerals['Age (Ma)'] = minerals['Age (Ma)'].fillna(value='Unknown') #  Changing the NaN values to unknown
minerals['Age (Ma)']

In [11]:
minerals['Age (Ma)'].isna().sum()
missing_values_count = minerals['Age (Ma)'].str.contains('Unknown', regex=False).value_counts()
total_cells = np.product(minerals['Age (Ma)'].shape)
total_missing = missing_values_count.sum()
percent_missing_age = (total_missing/total_cells) * 100
print('Number of missing cells: ', missing_values_count[:1])
print('% = ', percent_missing_age)

In [12]:
minerals_values_count = minerals.isnull().sum()
minerals_values_count

In [13]:
minerals['Age (Ma)'].value_counts()[0:25]
# Removing 1067 (18.6%) Age missing cells
age_removed = minerals[minerals['Age (Ma)'] != 'Unknown']
                                       
age_removed['Age (Ma)'].value_counts().sort_index(ascending=False)

In [14]:
# Creating a new dataframe for ploting
oldest_mine = age_removed.groupby('Mineral')[['Country', 'Age (Ma)']].max()
oldest_mine = oldest_mine.sort_values(by='Age (Ma)', ascending=False)[0:10]
oldest_min_notna = oldest_mine[~oldest_mine['Country'].str.contains('unknown')]
oldest_min_notna

# 4. Data Visualization

In [39]:
plt.figure(figsize=(20,5))
plt.title("Quantity of minerals per country")
sns.barplot(x=countries_splited['Country'].value_counts()[0:10].index, y=countries_splited['Country'].value_counts()[0:10])
plt.ylabel("Amount of minerals")
plt.xlabel("Countries", labelpad=14)

**Countries with more minerals:**
* United States of America: 825
* Russia: 803
* Italy: 370

In [16]:
plt.figure(figsize=(10,8))
a=minerals['Crystal Systems'].value_counts()[:30]
sns.countplot(data=minerals,x=a,palette='plasma')


In [17]:
countries_splited['Country'].value_counts()[0:10]
# Elements
# Merging countries_splited and elements_splited
countries_splited['Country'].value_counts()


In [18]:
elements_splited['Element'].value_counts()

In [19]:
filtro_1 = countries_splited['Country'].map(countries_splited['Country'].value_counts()) >= 200
elem_1 = countries_splited[filtro_1]
filtro_2 = elements_splited['Element'].map(elements_splited['Element'].value_counts()) >= 500
elem_2 = elements_splited[filtro_2]
plot_country_elem = pd.merge(elem_1, elem_2, on='Mineral')
plot_country_elem

In [20]:
plt.figure(figsize=(20,8))
plt.title("Most abundant chemistry element")
sns.lineplot(data=plot_country_elem.Element.value_counts())
plt.ylabel("Quantity")
plot_country_elem.Element.value_counts()

**Lover mass elements are more abundant in nature, for instance. Oxygen > Hydrogen > Sillica due to their higher activity of valence electrons**

In [21]:
# Edited from Daria Chemkaeva, (2020).
sns.catplot(x="Country", hue="Element", kind="count", palette="colorblind", edgecolor=".01", data=plot_country_elem, height=7, aspect=2)

> **Usa and Russia are the top two producers of minerals.**

**Most metals are found as types of rock in the Earth's crust. These ores contain sufficient minerals with important elements including metals that can be economically extracted from the rock. Metal ores are generally oxides, sulfides, silicates**

In [22]:
# Age
oldest_min = age_removed.groupby('Mineral')[['Country', 'Age (Ma)']].max()
oldest_min = oldest_min.sort_values(by='Age (Ma)', ascending=False)
oldest_min_notna = oldest_min[~oldest_min['Country'].str.contains('unknown')]
oldest_min_notna[0:20]

In [23]:
plt.figure(figsize=(15,5))
plt.title("Oldests minerals")
sns.barplot(x=oldest_min_notna[0:10].index, y=oldest_min_notna['Age (Ma)'][0:10])
plt.ylabel("Quantity")

In [24]:
plt.figure(figsize=(25,5))
plt.title("Countries with the oldest minerals")
sns.barplot(x=oldest_min_notna['Country'][0:10], y=oldest_min_notna['Age (Ma)'])
plt.ylabel("Age (Ma)")

In [25]:
age_group = oldest_min_notna.groupby('Mineral')[['Country', 'Age (Ma)']].max()
x = age_group.groupby(['Age (Ma)', 'Country'])['Age (Ma)'].unique()        #.apply(lambda df: df['Age (Ma)']).sort_values()
x.sort_values(ascending=False)


> **RESULTS**
1. Minerals provided by different countries. 

In [26]:
sns.set(font_scale=2)
sns.set_style("white")
plt.figure(figsize=(20,10))
plt.title("Quantity of minerals per country")
sns.barplot(x=countries_splited['Country'].value_counts()[0:10].index, y=countries_splited['Country'].value_counts()[0:10])
plt.ylabel("Amount of minerals")
plt.xlabel("Countries", labelpad=14)
countries_splited['Country'].value_counts()[0:15]


**Mineras provided by countries**
* United States of America: 825 minerals.
* Russia: 803 minerals.
* Italy: 370 minerals.

> 2. Most abundant chemistry element.

In [27]:
filtro_1 = countries_splited['Country'].map(countries_splited['Country'].value_counts()) >= 200
elem_1 = countries_splited[filtro_1]
filtro_2 = elements_splited['Element'].map(elements_splited['Element'].value_counts()) >= 500
elem_2 = elements_splited[filtro_2]
sns.set(font_scale=2)
sns.set_style("white")
plot_country_elem = pd.merge(elem_1, elem_2, on='Mineral')
sns.catplot(x="Country", hue="Element", kind="count", palette="colorblind", edgecolor=".01", data=plot_country_elem, height=7, aspect=4)


In [28]:
# * United States of America has more elements, since they are the country with more minerals
sns.set(font_scale=2)
sns.set_style("white")
plt.figure(figsize=(35,10))
plt.title("Most abundant chemistry element")
sns.lineplot(data=plot_country_elem.Element.value_counts())
plt.ylabel("Quantity")
plot_country_elem.Element.value_counts()

> 3. Oldest mineral.

In [29]:
sns.set(font_scale=2)
sns.set_style("white")
plt.figure(figsize=(25,5))
plt.title("Oldests minerals")
sns.barplot(x=oldest_min_notna[0:10].index, y=oldest_min_notna['Age (Ma)'][0:10])
plt.ylabel("Quantity")

> This are the oldest minerals in the data, with an age of 4.7 Ma:
 
1. Tochilinite
2. Machiite
3. Troilite
4. Forsterite
5. Warkite
6. Aragonite
7. Pentlandite
8. Perovskite
9. Corundum
10. Magnetite

In [30]:
sns.set(font_scale=2)
plt.figure(figsize=(25,5))
plt.title("Countries with the oldest minerals")
sns.barplot(x=oldest_min_notna['Country'][0:5], y=oldest_min_notna['Age (Ma)'])
plt.ylabel("Age (Ma)")

> Countries having the oldest minerals in the data:

 **Russia,Australia,Italy (Meteorite),Italy,Australia (Meteorite)/ Italy (Meteorite)**

In [31]:
sns.set(font_scale=2)
plt.figure(figsize=(35,15))
plt.title("Most abundant age")
sns.lineplot(data=oldest_min_notna['Age (Ma)'].value_counts())
plt.ylabel("Quantity")
oldest_min_notna['Age (Ma)'].value_counts(ascending=False)

In [32]:
sns.set(font_scale=2)
sns.set_style("white")
plt.figure(figsize=(35,15))
sns.distplot(a=oldest_min_notna['Age (Ma)'], kde=True)
plt.title("Histogram of Age (Ma)")
plt.legend()

In [33]:
# The Age (Ma) values spread from 0<x<4700 Ma. Most of the values are between 0 and 600 Ma.
sns.set_style(style="white")
rs = np.random.RandomState(5000)
new_data = pd.DataFrame(data=rs.normal(size=(100, 14)), columns=list(minerals.columns))
corr = new_data.corr()
matrix = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(7, 10))
cmap = sns.diverging_palette(10, 100, as_cmap=True)
sns.heatmap(corr, mask=matrix, cmap=cmap, vmax=.1, center=0, square=True, cbar_kws={"shrink": .50})

# 5.Summary 
More than 4,000 naturally occurring minerals—inorganic solids that have a characteristic chemical composition and specific crystal structure—have been found on Earth(Forsterite, Tochilinite, Machiite, Troilite, Troilite Warkite, Aragonite, Pentlandite, Perovskite, Corona, and Magnetite are the oldest minerals in the data with ages of 4.7 Ma.)
Russia and the United States are the top two mineral producers. Russia donated 803 minerals, the United States of America provided 825 minerals, and Italy contributed 370 minerals.
